In [2]:
import pickle
import h5py
from openfermion import FermionOperator, get_majorana_operator

In [3]:
with h5py.File("h2_commutator_tensors.h5", "r") as fid:

    # Access the "BLISS_HAM" group
    mol_data_group = fid["BLISS_HAM"]
    
    # Read multi-dimensional data
    h_const = mol_data_group["h_const"][:]
    obt = mol_data_group["obt"][:]
    tbt = mol_data_group["tbt"][:]
    threebt = mol_data_group["threebt"][:]
    eta = mol_data_group["eta"][()]

In [4]:
print(tbt)

[[[[ 0.81176116  0.        ]
   [ 0.          0.41817999]]

  [[ 0.         -1.0701146 ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [-1.0701146   0.        ]]

  [[ 0.41817999  0.        ]
   [ 0.          0.02459882]]]]


In [5]:
def construct_one_body_fermion_operator(ferm_op, obt):

    num_orbitals = obt.shape[0]  # assuming h1e is square
    num_spin_orbs = 2 * num_orbitals
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            coefficient = obt[p, q]
            if coefficient != 0.0:
                
                term_alpha_alpha = FermionOperator(((2*p, 1), (2*q, 0)), coefficient)
                ferm_op += term_alpha_alpha
    
                # Beta-beta: (2*p+1, 1) and (2*q+1, 0)
                term_beta_beta = FermionOperator(((2*p+1, 1), (2*q+1, 0)), coefficient)
                ferm_op += term_beta_beta
    
    return ferm_op


def construct_two_body_fermion_operator(ferm_op, tbt):

    num_orbitals = tbt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    coefficient = tbt[p, q, r, s]
                    if coefficient != 0.0:
                        term_1 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_2 = FermionOperator(((2*p + 1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0)), coefficient)
                        term_3 = FermionOperator(((2*p, 1), (2*q, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        term_4 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r + 1, 1), (2*s + 1, 0)), coefficient)
                        
                        ferm_op += term_1 + term_2 + term_3 + term_4
    
    return ferm_op

def construct_three_body_fermion_operator(ferm_op, threebt):
    num_orbitals = threebt.shape[0]  # assuming h1e is square
    for p in range(num_orbitals):
        for q in range(num_orbitals):
            for r in range(num_orbitals):
                for s in range(num_orbitals):
                    for m in range(num_orbitals):
                        for n in range(num_orbitals):
                            coefficient = threebt[p, q, r, s, m, n]
                            if coefficient != 0.0:
                                term_1 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_2 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_3 = FermionOperator(((2*p, 1), (2*q, 0), (2*r+1, 1), (2*s+1, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_4 = FermionOperator(((2*p, 1), (2*q, 0), (2*r, 1), (2*s, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_5 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r+1, 1), (2*s+1, 0), (2*m, 1), (2*n, 0)), coefficient)
                                term_6 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r, 1), (2*s, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_7 = FermionOperator(((2*p, 1), (2*q, 0), (2*r+1, 1), (2*s+1, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)
                                term_8 = FermionOperator(((2*p+1, 1), (2*q+1, 0), (2*r+1, 1), (2*s+1, 0), (2*m+1, 1), (2*n+1, 0)), coefficient)


                                ferm_op += term_1 + term_2 + term_3 + term_4 + term_5 + term_6 + term_7 + term_8
    
    return ferm_op

In [6]:
fermion_operator = FermionOperator()
fermion_operator = construct_one_body_fermion_operator(fermion_operator, obt)
fermion_operator = construct_two_body_fermion_operator(fermion_operator, tbt)
fermion_operator = construct_three_body_fermion_operator(fermion_operator, threebt)

In [7]:
print(fermion_operator)

-3.2470446275102587 [0^ 0] +
0.8117611568775646 [0^ 0 0^ 0] +
0.8117611568775646 [0^ 0 1^ 1] +
0.4181799899066242 [0^ 0 2^ 2] +
0.4181799899066242 [0^ 0 3^ 3] +
-1.0701145997683443 [0^ 2 0^ 2] +
-1.0701145997683443 [0^ 2 1^ 3] +
-3.2470446275102587 [1^ 1] +
0.8117611568775646 [1^ 1 0^ 0] +
0.8117611568775646 [1^ 1 1^ 1] +
0.4181799899066242 [1^ 1 2^ 2] +
0.4181799899066242 [1^ 1 3^ 3] +
-1.0701145997683443 [1^ 3 0^ 2] +
-1.0701145997683443 [1^ 3 1^ 3] +
-1.0701145997683443 [2^ 0 2^ 0] +
-1.0701145997683443 [2^ 0 3^ 1] +
-0.09839529174273509 [2^ 2] +
0.4181799899066242 [2^ 2 0^ 0] +
0.3935811669709405 [2^ 2 0^ 0 0^ 0] +
0.3935811669709405 [2^ 2 0^ 0 1^ 1] +
0.4181799899066242 [2^ 2 1^ 1] +
0.3935811669709405 [2^ 2 1^ 1 0^ 0] +
0.3935811669709405 [2^ 2 1^ 1 1^ 1] +
0.024598822935683773 [2^ 2 2^ 2] +
-0.3935811669709405 [2^ 2 2^ 2 0^ 0] +
-0.3935811669709405 [2^ 2 2^ 2 1^ 1] +
0.024598822935683773 [2^ 2 3^ 3] +
-0.3935811669709405 [2^ 2 3^ 3 0^ 0] +
-0.3935811669709405 [2^ 2 3^ 3 1^ 1] +


In [11]:
with open(f'SolvableQubitHamiltonians/commutator_store/h2_commutator_blissed.pkl', 'wb') as f:
    pickle.dump(fermion_operator, f)

In [1]:
print(fermion_operator)

NameError: name 'fermion_operator' is not defined

In [8]:
majo = get_majorana_operator(fermion_operator)
    
one_norm = 0
for term, coeff in majo.terms.items():
    one_norm += abs(coeff)
print(one_norm)

6.592616150895454
